In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

https://www.census.gov/topics/public-sector/voting/data/tables.2010.List_1863097513.html#list-tab-List_1863097513

In [2]:
# read in the Excel file and specify the sheet name
df_votes_2020 = pd.read_excel('Election_Results/vote2020.xlsx', sheet_name='Table 4a', header=[4])

# display the resulting dataframe
df_votes_2020.head()

Table 4a


STATE  Total population  Total citizen population   
0            NaN               NaN                       NaN  \
1  UNITED STATES          252274.0                  231593.0   
2        ALABAMA            3769.0                    3716.0   
3         ALASKA             528.0                     516.0   
4        ARIZONA            5638.0                    5075.0   

         Registered                   Unnamed: 4         Unnamed: 5   
0  Total registered  Percent registered\n(Total)  Margin of error 1  \
1            168308                         66.7                0.4   
2              2527                           67                3.1   
3               383                         72.6                3.2   
4              3878                         68.8                2.5   

                      Unnamed: 6         Unnamed: 7        Voted   
0  Percent registered\n(Citizen)  Margin of error 1  Total voted  \
1                           72.7                0.4       154628   
2                             68                3.1         2247   
3                           74.2                3.1          330   
4                           76.4                2.5         3649   

               Unnamed: 9        Unnamed: 10               Unnamed: 11   
0  Percent voted\n(Total)  Margin of error 1  Percent voted\n(Citizen)  \
1                    61.3                0.4                      66.8   
2                    59.6                3.3                      60.5   
3                    62.4                3.4                      63.8   
4                    64.7                2.6                      71.9   

         Unnamed: 12  
0  Margin of error 1  
1                0.4  
2                3.3  
3                3.4  
4                2.6

In [3]:
# select the columns wanted and rename them
df_selection = df_votes_2020[['STATE', 'Total population', 'Registered', 'Voted']]
df_votes = df_selection.rename(columns={
    'STATE': 'State',
    'Total population': 'Population',
    'Registered': 'Registered'
})
# add year and office columns
df_votes['Year'] = 2020

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'State', 'Population', 'Registered', 'Voted']

df_votes = df_votes.iloc[1:]  # remove the first row
df_votes = df_votes.iloc[:-7] # removes last 7 rows of text

# Save the first row as a separate DataFrame
first_row = df_votes.iloc[[0]].copy()

# Remove the first row from the original DataFrame
df_votes = df_votes.iloc[1:].copy()

# Append the first row to the end of the DataFrame
df_votes = pd.concat([df_votes, first_row])

df_votes = df_votes.reset_index(drop=True)  # reset the index
df_votes = df_votes.reindex(columns=new_cols) # set year column in front

df_votes
# values are in the thousands

Year                 State  Population Registered   Voted
0   2020               ALABAMA      3769.0       2527    2247
1   2020                ALASKA       528.0        383     330
2   2020               ARIZONA      5638.0       3878    3649
3   2020              ARKANSAS      2283.0       1361    1186
4   2020            CALIFORNIA     30342.0      18001   16893
5   2020              COLORADO      4525.0       2993    2837
6   2020           CONNECTICUT      2777.0       1850    1681
7   2020              DELAWARE       766.0        542     489
8   2020  DISTRICT OF COLUMBIA       576.0        464     448
9   2020               FLORIDA     17244.0      10495    9720
10  2020               GEORGIA      8032.0       5233    4888
11  2020                HAWAII      1056.0        673     630
12  2020                 IDAHO      1370.0        900     843
13  2020              ILLINOIS      9658.0       6590    6058
14  2020               INDIANA      5096.0       3412    3002
15  2020                  IOWA      2361.0       1742    1618
16  2020                KANSAS      2157.0       1398    1297
17  2020              KENTUCKY      3384.0       2450    2210
18  2020             LOUISIANA      3438.0       2286    2041
19  2020                 MAINE      1087.0        832     766
20  2020              MARYLAND      4606.0       3383    3166
21  2020         MASSACHUSETTS      5514.0       3546    3249
22  2020              MICHIGAN      7790.0       5513    4994
23  2020             MINNESOTA      4339.0       3436    3225
24  2020           MISSISSIPPI      2212.0       1749    1531
25  2020              MISSOURI      4637.0       3388    2990
26  2020               MONTANA       836.0        641     607
27  2020              NEBRASKA      1435.0        971     892
28  2020                NEVADA      2402.0       1455    1351
29  2020         NEW HAMPSHIRE      1101.0        843     797
30  2020            NEW JERSEY      6801.0       5008    4638
31  2020            NEW MEXICO      1610.0       1028     938
32  2020              NEW YORK     15105.0       9370    8609
33  2020        NORTH CAROLINA      8113.0       5161    4780
34  2020          NORTH DAKOTA       571.0        429     373
35  2020                  OHIO      8951.0       6733    6128
36  2020              OKLAHOMA      2942.0       1884    1631
37  2020                OREGON      3369.0       2590    2402
38  2020          PENNSYLVANIA      9902.0       7337    6756
39  2020          RHODE ISLAND       840.0        575     515
40  2020        SOUTH CAROLINA      4010.0       2713    2459
41  2020          SOUTH DAKOTA       659.0        437     380
42  2020             TENNESSEE      5283.0       3742    3346
43  2020                 TEXAS     21485.0      13343   11874
44  2020                  UTAH      2320.0       1468    1386
45  2020               VERMONT       507.0        365     342
46  2020              VIRGINIA      6481.0       4541    4275
47  2020            WASHINGTON      5993.0       4029    3854
48  2020         WEST VIRGINIA      1397.0        928     773
49  2020             WISCONSIN      4538.0       3391    3253
50  2020               WYOMING       436.0        296     280
51  2020         UNITED STATES    252274.0     168308  154628

In [4]:
state_abbr = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
    'UNITED STATES': 'US'
}



In [5]:
df_votes['State'] = df_votes['State'].map(state_abbr)
df_votes

Year State  Population Registered   Voted
0   2020    AL      3769.0       2527    2247
1   2020    AK       528.0        383     330
2   2020    AZ      5638.0       3878    3649
3   2020    AR      2283.0       1361    1186
4   2020    CA     30342.0      18001   16893
5   2020    CO      4525.0       2993    2837
6   2020    CT      2777.0       1850    1681
7   2020    DE       766.0        542     489
8   2020    DC       576.0        464     448
9   2020    FL     17244.0      10495    9720
10  2020    GA      8032.0       5233    4888
11  2020    HI      1056.0        673     630
12  2020    ID      1370.0        900     843
13  2020    IL      9658.0       6590    6058
14  2020    IN      5096.0       3412    3002
15  2020    IA      2361.0       1742    1618
16  2020    KS      2157.0       1398    1297
17  2020    KY      3384.0       2450    2210
18  2020    LA      3438.0       2286    2041
19  2020    ME      1087.0        832     766
20  2020    MD      4606.0       3383    3166
21  2020    MA      5514.0       3546    3249
22  2020    MI      7790.0       5513    4994
23  2020    MN      4339.0       3436    3225
24  2020    MS      2212.0       1749    1531
25  2020    MO      4637.0       3388    2990
26  2020    MT       836.0        641     607
27  2020    NE      1435.0        971     892
28  2020    NV      2402.0       1455    1351
29  2020    NH      1101.0        843     797
30  2020    NJ      6801.0       5008    4638
31  2020    NM      1610.0       1028     938
32  2020    NY     15105.0       9370    8609
33  2020    NC      8113.0       5161    4780
34  2020    ND       571.0        429     373
35  2020    OH      8951.0       6733    6128
36  2020    OK      2942.0       1884    1631
37  2020    OR      3369.0       2590    2402
38  2020    PA      9902.0       7337    6756
39  2020    RI       840.0        575     515
40  2020    SC      4010.0       2713    2459
41  2020    SD       659.0        437     380
42  2020    TN      5283.0       3742    3346
43  2020    TX     21485.0      13343   11874
44  2020    UT      2320.0       1468    1386
45  2020    VT       507.0        365     342
46  2020    VA      6481.0       4541    4275
47  2020    WA      5993.0       4029    3854
48  2020    WV      1397.0        928     773
49  2020    WI      4538.0       3391    3253
50  2020    WY       436.0        296     280
51  2020    US    252274.0     168308  154628

In [6]:
# Calculate the normalized voter count
df_votes['Norm_Voter_Reg'] = df_votes['Voted'] / df_votes['Registered']
df_votes['Norm_Voter_Pop'] = df_votes['Voted'] / df_votes['Population']
df_votes

Year State  Population Registered   Voted Norm_Voter_Reg Norm_Voter_Pop
0   2020    AL      3769.0       2527    2247       0.889197       0.596179
1   2020    AK       528.0        383     330       0.861619          0.625
2   2020    AZ      5638.0       3878    3649       0.940949       0.647215
3   2020    AR      2283.0       1361    1186       0.871418       0.519492
4   2020    CA     30342.0      18001   16893       0.938448       0.556753
5   2020    CO      4525.0       2993    2837       0.947878       0.626961
6   2020    CT      2777.0       1850    1681       0.908649       0.605329
7   2020    DE       766.0        542     489       0.902214       0.638381
8   2020    DC       576.0        464     448       0.965517       0.777778
9   2020    FL     17244.0      10495    9720       0.926155       0.563674
10  2020    GA      8032.0       5233    4888       0.934072       0.608566
11  2020    HI      1056.0        673     630       0.936107       0.596591
12  2020    ID      1370.0        900     843       0.936667       0.615328
13  2020    IL      9658.0       6590    6058       0.919272       0.627252
14  2020    IN      5096.0       3412    3002       0.879836       0.589089
15  2020    IA      2361.0       1742    1618       0.928817       0.685303
16  2020    KS      2157.0       1398    1297       0.927754       0.601298
17  2020    KY      3384.0       2450    2210       0.902041       0.653073
18  2020    LA      3438.0       2286    2041       0.892826       0.593659
19  2020    ME      1087.0        832     766       0.920673       0.704692
20  2020    MD      4606.0       3383    3166       0.935856       0.687364
21  2020    MA      5514.0       3546    3249       0.916244       0.589227
22  2020    MI      7790.0       5513    4994       0.905859       0.641078
23  2020    MN      4339.0       3436    3225       0.938591       0.743259
24  2020    MS      2212.0       1749    1531       0.875357       0.692134
25  2020    MO      4637.0       3388    2990       0.882527       0.644813
26  2020    MT       836.0        641     607       0.946958       0.726077
27  2020    NE      1435.0        971     892       0.918641       0.621603
28  2020    NV      2402.0       1455    1351       0.928522       0.562448
29  2020    NH      1101.0        843     797       0.945433       0.723887
30  2020    NJ      6801.0       5008    4638       0.926118       0.681959
31  2020    NM      1610.0       1028     938       0.912451       0.582609
32  2020    NY     15105.0       9370    8609       0.918783       0.569944
33  2020    NC      8113.0       5161    4780       0.926177       0.589178
34  2020    ND       571.0        429     373       0.869464        0.65324
35  2020    OH      8951.0       6733    6128       0.910144       0.684616
36  2020    OK      2942.0       1884    1631       0.865711       0.554385
37  2020    OR      3369.0       2590    2402       0.927413       0.712971
38  2020    PA      9902.0       7337    6756       0.920812       0.682286
39  2020    RI       840.0        575     515       0.895652       0.613095
40  2020    SC      4010.0       2713    2459       0.906377       0.613217
41  2020    SD       659.0        437     380       0.869565       0.576631
42  2020    TN      5283.0       3742    3346       0.894174       0.633352
43  2020    TX     21485.0      13343   11874       0.889905       0.552665
44  2020    UT      2320.0       1468    1386       0.944142       0.597414
45  2020    VT       507.0        365     342       0.936986       0.674556
46  2020    VA      6481.0       4541    4275       0.941423        0.65962
47  2020    WA      5993.0       4029    3854       0.956565       0.643084
48  2020    WV      1397.0        928     773       0.832974       0.553329
49  2020    WI      4538.0       3391    3253       0.959304       0.716836
50  2020    WY       436.0        296     280       0.945946       0.642202
51  2020    US    252274.0     168308  154628       

# Add to sql db

In [7]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the results table
df_votes_2020.to_sql("voters", engine, if_exists="replace", index=False)
print("Voters table loaded successfully")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


Voters table loaded successfully


In [8]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM voters", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()


    Year State  Population  Registered   Voted  Norm_Voter_Reg  Norm_Voter_Pop
0   2020    AL      3769.0        2527    2247        0.889197        0.596179
1   2020    AK       528.0         383     330        0.861619        0.625000
2   2020    AZ      5638.0        3878    3649        0.940949        0.647215
3   2020    AR      2283.0        1361    1186        0.871418        0.519492
4   2020    CA     30342.0       18001   16893        0.938448        0.556753
5   2020    CO      4525.0        2993    2837        0.947878        0.626961
6   2020    CT      2777.0        1850    1681        0.908649        0.605329
7   2020    DE       766.0         542     489        0.902214        0.638381
8   2020    DC       576.0         464     448        0.965517        0.777778
9   2020    FL     17244.0       10495    9720        0.926155        0.563674
10  2020    GA      8032.0        5233    4888        0.934072        0.608566
11  2020    HI      1056.0         673     630      